# [Advent of Code 2020 Day ?]()

?

## Initial setup

In [1]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

In [2]:
from abc import ABC, abstractmethod

class Grammar:
    def __init__(self):
        self.database: dict[int, Rule] = {}
        self.raw_rules: list[str] = []
        self.inputs: list[str] = []

    def add_input(self, data: str):
        self.inputs.append(data)

    def add_rule(self, data: str):
        assert (groups := parse(r"^(\d+): (.*)$", data)) is not None, f"Could not parse {data}"
        assert int(groups[0]) not in self.database, f"Attempted to add rule number {groups[0]} with {data=} into database even though rule number already exists"
        self.raw_rules.append(data)
        self.database[int(groups[0])] = Rule.from_string(groups[1], self.database)

    def match(self, other: str) -> bool:
        for offset in self.database[0].match(other):
            if offset == len(other):
                return True
        return False

class Rule(ABC):

    @abstractmethod
    def __init__(self, data: str, database: dict[int, "Rule"]) -> None:
        raise NotImplementedError

    @abstractmethod
    def match(self, other: str) -> list[int]:
        raise NotImplementedError

    @staticmethod
    def from_string(data: str, database: dict[int, "Rule"]) -> "Rule":
        try:
            return TerminalRule(data, database)
        except AssertionError:
            pass
        try:
            return NonTerminalRule(data, database)
        except AssertionError:
            pass
        try:
            return UnionRule(data, database)
        except AssertionError:
            pass
        raise ValueError(f"Could not find any rule to match for {data}")

class TerminalRule(Rule):

    def __init__(self, data: str, database: dict[int, Rule]):
        assert (groups := parse(r"^\"([a-z]+)\"$", data)) is not None, f"Couldn't parse {data} for TerminalRule"
        assert len(groups) == 1, f"Groups for TerminalRule {data} should be 1 but got {len(groups)} => {groups}"
        self.database = database
        self.terminal = groups[0]

    def match(self, other: str) -> list[int]:
        if other.startswith(self.terminal) or other == self.terminal:
            return [len(self.terminal)]
        return []

class NonTerminalRule(Rule):

    def __init__(self, data: str, database: dict[int, Rule]):
        assert (groups := parse(r"^((?:\d+ ?)+)$", data)), f"Couldn't parse {data} for NonTerminalRule"
        assert len(groups) == 1, f"Groups for NonTerminalRule {data} should be 1 but got {len(groups)} => {groups}"
        self.database = database
        self.rules = list(map(int, groups[0].split()))

    @staticmethod
    def backtrack(database: dict[int, Rule], rules: list[int], idx: int, other: str, curr_offset: int, matched_offsets: list[int]) -> None:

        if idx >= len(rules):
            matched_offsets.append(curr_offset)
            return

        rule = database[rules[idx]]

        if (munches := rule.match(other)) is None:
            return

        for offset in munches:
            NonTerminalRule.backtrack(database, rules, idx + 1, other[offset:], curr_offset + offset, matched_offsets)

    def match(self, other: str) -> list[int]:
        offsets: list[int] = []
        NonTerminalRule.backtrack(self.database, self.rules, 0, other, 0, offsets)
        return offsets

class UnionRule(Rule):

    def __init__(self, data: str, database: dict[int, Rule]):
        assert (groups := parse(r"^((?:\d+ ?)+) \| ((?:\d+ ?)+)$", data)), f"Couldn't parse {data} for UnionRule"
        assert len(groups) == 2, f"Groups for UnionRule {data} should be 2 but got {len(groups)} => {groups}"
        self.database = database
        self.choice_1 = NonTerminalRule(groups[0], self.database)
        self.choice_2 = NonTerminalRule(groups[1], self.database)

    def match(self, other: str) -> list[int]:
        return self.choice_1.match(other) + self.choice_2.match(other)

## Input Parsing

In [3]:
def parse_input(filename: str) -> Any:

    gen = yield_line(filename)

    grammar = Grammar()

    for line in gen:
        if line == "":
            break
        grammar.add_rule(line)

    for line in gen:
        grammar.add_input(line)

    return grammar

Some tests for the classes I made...

In [4]:
%%ipytest

grammar: Grammar = parse_input("example1")

def test_rule_instantiation():
    assert isinstance(grammar.database[0], NonTerminalRule)
    assert isinstance(grammar.database[1], UnionRule)
    assert isinstance(grammar.database[2], UnionRule)
    assert isinstance(grammar.database[3], UnionRule)
    assert isinstance(grammar.database[4], TerminalRule)
    assert isinstance(grammar.database[5], TerminalRule)

def test_grammar_terminals():
    assert grammar.database[4].match("a")
    assert grammar.database[5].match("b")

def test_grammar_union_rule_simple():
    assert grammar.database[3].match("ab")
    assert grammar.database[3].match("ba")
    assert not grammar.database[3].match("qq")
    assert not grammar.database[3].match("aa")
    assert not grammar.database[3].match("bb")

def test_grammar_start_rule_match():
    assert grammar.database[0].match("ababbb")
    assert grammar.database[0].match("abbbab")

def test_grammar_start_rule_no_match():
    assert not grammar.database[0].match("bababa")
    assert not grammar.database[0].match("aaabbb")
    assert grammar.database[0].match("aaaabbb") == [6]  # the low-level match functions only return offsets; it is up to the Grammar class to determine that leftover letters = Fail

.....                                                                                        [100%]
5 passed in 0.01s


## Part 1
Lorem ipsum

In [5]:
def part_one(grammar: Grammar) -> int | str:
    match_count: int = 0
    for input_line in grammar.inputs:
        if grammar.match(input_line):
            match_count += 1
    return match_count

In [6]:
%%ipytest
def test_part_one():
    assert part_one(parse_input("example1")) == 2
    assert part_one(parse_input("input")) == 126

.                                                                                            [100%]
1 passed in 0.05s


## Part 2
Lorem ipsum

In [7]:
def part_two(data: Any) -> int | str:
    return 0x3f3f3f3f + 2

In [8]:
%%ipytest
def test_part_two():
    assert part_two(parse_input("example1")) == 0x3f3f3f3f + 2
    assert part_two(parse_input("input")) == 0x3f3f3f3f + 2

.                                                                                            [100%]
1 passed in 0.01s
